In [1]:
import requests

In [2]:
standings_url = "https://www.pro-football-reference.com/years/2023/index.htm"

In [3]:
data = requests.get(standings_url)

In [4]:
from bs4 import BeautifulSoup

In [5]:
soup = BeautifulSoup(data.text)

In [6]:
afc_standings_table = soup.select('table.stats_table')[0]

In [7]:
nfc_standings_table = soup.select('table.stats_table')[1]

In [8]:
afc_links = afc_standings_table.find_all('a')
nfc_links = nfc_standings_table.find_all('a')

In [9]:
nfl_links = afc_links + nfc_links

In [10]:
nfl_links = [l.get("href") for l in nfl_links]

In [11]:
nfl_links = [l for l in nfl_links if '/teams/' in l]

In [12]:
nfl_urls = [f"https://www.pro-football-reference.com{l}" for l in nfl_links]

In [13]:
team_url = nfl_urls[0]

In [14]:
team_url.split("/")[-2]

'buf'

In [15]:
data = requests.get(team_url)

In [16]:
import pandas as pd

matches = pd.read_html(data.text, match="Schedule & Game Results")[0]

In [17]:
matches.columns = matches.columns.droplevel()

In [18]:
years = list(range(2024, 2021, -1))
all_matches = []

In [19]:
standings_url = "https://www.pro-football-reference.com/years/2024/index.htm"

In [20]:
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    afc_standings_table = soup.select('table.stats_table')[0]
    nfc_standings_table = soup.select('table.stats_table')[1]
    
    afc_links = afc_standings_table.find_all('a')
    nfc_links = nfc_standings_table.find_all('a')
    nfl_links = afc_links + nfc_links
    nfl_links = [l.get("href") for l in nfl_links]
    nfl_links = [l for l in nfl_links if '/teams/' in l]
    nfl_urls = [f"https://www.pro-football-reference.com{l}" for l in nfl_links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://www.pro-football-reference.com{previous_season}"
    
    for team_url in nfl_urls:
        team_name = team_url.split("/")[-2]
        
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Schedule & Game Results")[0]
        
        team_data = matches
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(1)
        

In [21]:
match_df = pd.concat(all_matches)

In [22]:
match_df

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0  \
                 Week                Day               Date   
0                   1                Sun        September 8   
1                   2                Thu       September 12   
2                   3                Mon       September 23   
3                   4                Sun       September 29   
4                   5                Sun          October 6   
..                ...                ...                ...   
13                 14                Mon        December 12   
14                 15                Sun        December 18   
15                 16                Sun        December 25   
16                 17                Sun          January 1   
17                 18                Sun          January 8   

   Unnamed: 3_level_0 Unnamed: 4_level_0 Unnamed: 5_level_0  \
   Unnamed: 3_level_1 Unnamed: 4_level_1 Unnamed: 5_level_1   
0           1:00PM ET           boxscore                  W   
1           8:15PM ET           boxscore                  W   
2           7:30PM ET           boxscore                NaN   
3           8:20PM ET            preview                NaN   
4           1:00PM ET            preview                NaN   
..                ...                ...                ...   
13          8:15PM ET           boxscore                  L   
14          4:05PM ET           boxscore                  L   
15          8:20PM ET           boxscore                  L   
16          1:00PM ET           boxscore                  L   
17          4:25PM ET           boxscore                  L   

   Unnamed: 6_level_0 Unnamed: 7_level_0 Unnamed: 8_level_0  \
                   OT                Rec Unnamed: 8_level_1   
0                 NaN                1-0                NaN   
1                 NaN                2-0                  @   
2                 NaN                NaN                NaN   
3                 NaN                NaN                  @   
4                 NaN                NaN                  @   
..                ...                ...                ...   
13                NaN                4-9                NaN   
14                NaN               4-10                  @   
15                 OT               4-11                NaN   
16                NaN               4-12                  @   
17                NaN               4-13                  @   

      Unnamed: 9_level_0  ... Defense                            \
                     Opp  ...    1stD  TotYd  PassY  RushY   TO   
0      Arizona Cardinals  ...    18.0  270.0  146.0  124.0  1.0   
1         Miami Dolphins  ...    20.0  351.0  212.0  139.0  3.0   
2   Jacksonville Jaguars  ...     NaN    NaN    NaN    NaN  NaN   
3       Baltimore Ravens  ...     NaN    NaN    NaN    NaN  NaN   
4         Houston Texans  ...     NaN    NaN    NaN    NaN  NaN   
..                   ...  ...     ...    ...    ...    ...  ...   
13  New England Patriots  ...    18.0  328.0  225.0  103.0  1.0   
14        Denver Broncos  ...    23.0  324.0  156.0  168.0  2.0   
15  Tampa Bay Buccaneers  ...    21.0  396.0  281.0  115.0  2.0   
16       Atlanta Falcons  ...    20.0  298.0  166.0  132.0  1.0   
17   San Francisco 49ers  ...    21.0  311.0  142.0  169.0  NaN   

   Expected Points                 Season Team  
           Offense Defense Sp. Tms              
0            13.51   -3.22   -2.25   2024  buf  
1             8.59   14.11   -2.53   2024  buf  
2              NaN     NaN     NaN   2024  buf  
3              NaN     NaN     NaN   2024  buf  
4              NaN     NaN     NaN   2024  buf  
..             ...     ...     ...    ...  ...  
13          -17.43    0.47     0.9   2022  crd  
14          -19.01    0.79     6.6   2022  crd  
15          -13.64     4.6    6.77   2022  crd  
16             6.1   -3.27    -5.1   2022  crd  
17          -15.23   -9.62    -0.8   2022  crd  

[1808 rows x 27 columns]

In [23]:
match_df.columns = [c.lower() for c in match_df.columns]

AttributeError: 'tuple' object has no attribute 'lower'

In [24]:
match_df.to_csv("macthes.csv")